# Tokenization

### BertTweet

- fastBPE
- 64K subword

### Twilbert
- SentencePiece (fastBPE)
- 30k subword 

In [1]:
%load_ext autoreload
%autoreload 2
from glob import glob

num_files = 100
tweet_files = glob("../data/filtered_tweets/*.txt")

train_files = tweet_files[:2]


tweets = list([x.strip("\n") for x in open("../data/filtered_tweets/spanish-tweets-000.txt")])[:100_000]

In [2]:
len(tweets)

100000

In [3]:
from tokenizers import SentencePieceUnigramTokenizer, SentencePieceBPETokenizer, BertWordPieceTokenizer

tokenizer = SentencePieceBPETokenizer()

In [4]:
from finetune_vs_scratch.preprocessing import special_tokens
from finetune_vs_scratch.tokenizer import tokenizer_special_tokens

#tokenizer.add_special_tokens(tokenizer_special_tokens)
tokenizer.train_from_iterator(
    tweets,
    vocab_size=30_000,
    min_frequency=5,
    show_progress=True,
    special_tokens=tokenizer_special_tokens,
    limit_alphabet=500,
)

In [5]:

vocab = tokenizer.get_vocab()
inv_vocab = {v:k for k, v in vocab.items()}


In [6]:
vocab["@usuario"]

525

In [7]:

tokenizer.encode("Qué hacesssss @usuario").tokens

['▁Qué', '▁haces', 'ss', 'ss', '▁@usuario']

In [8]:


" ".join([inv_vocab[i] for i in range(1000)])

'<s> </s> <unk> <pad> <mask> ! " # $ % & \' ( ) * + , - . / 0 1 2 3 4 5 6 7 8 9 : ; = ? @ A B C D E F G H I J K L M N O P Q R S T U V W X Y Z [ \\ ] ^ _ ` a b c d e f g h i j k l m n o p q r s t u v w x y z { | } ¡ ¿ À Á È É Í Ñ Ó Ø Ú Ü à á â ã ä å ç è é ê ì í ï ñ ò ó ô õ ö ø ù ú ü ō ɜ ɪ ɴ ɾ ʀ ʏ ʔ ʕ ʝ ˎ ˶ ́ ̄ ̆ ̓ ̟ ̡ ̣ ̤ ̳ ̶ α ι ρ ω Д а е з и т א ر ل م و ي ٩ ۶ ಥ ฅ ๑ ყ ძ ხ ᄏ ᅲ ᥆ ᥉ ᥒ ᥙ ᥣ ᥱ ᥲ ᥴ ᴀ ᴅ ᴇ ᴏ ᴖ ᴗ ᴘ ᴛ ᴜ \u200b \u200d ⁄ \u2066 \u2069 ⃣ ↑ → ↓ ⇒ ∇ ∧ ≦ ≧ ⊙ ⌓ ⏱ ─ ━ ┃ ┏ ┓ ┗ ┛ ═ ║ ╗ ╚ ╭ ╮ ▁ █ □ ▪ ▶ ► ▽ ◇ ○ ◍ ☀ ★ ☆ ☑ ☟ ♪ ♫ ⚠ ✍ ✏ ✔ ❣ ➡ ⠀ ⬆ ⬇ ⬈ 、 。 「 」 『 』 【 】 〣 あ い う え お か が き く け こ ご さ し す せ そ た だ ち っ つ づ て で と ど な に の は ひ び ま み め も や よ ら り る れ ろ わ を ん ゙ ゚ ァ ア ィ イ ウ ェ オ カ キ ク グ コ サ シ ジ ス ズ タ チ ッ ツ ト ド ナ ニ ノ バ パ フ プ ベ ボ マ ミ ム メ モ ャ ュ ユ ラ リ ル レ ロ ン ・ ヾ 一 三 中 予 人 今 付 作 你 催 全 公 前 午 去 叉 可 喔 在 場 売 夜 大 好 始 定 川 巻 年 愛 描 放 新 日 是 時 月 本 森 欸 正 浪 海 無 版 特 生 用 画 発 登 看 真 神 約 紙 細 色 華 詳 誕 豪 賊 送 逆 開 零 音 ꧟ 가 거 게 고 기 나 남 년 니 다 단 대 더 들 라 랑 리 마 면 무 미 민 방 뷔 사 생 서 소 수 슈 스 시 아 야 어 에 예 요 우 은 을 이 일 자 정 제 주 준 지 짐 탄 태

In [9]:
tokenizer_path = "./sentence-piece-tokenizer"
!mkdir $tokenizer_path
vocab_file, merges_file = tokenizer.save_model(tokenizer_path)

mkdir: cannot create directory ‘./sentence-piece-tokenizer’: File exists


In [10]:

from finetune_vs_scratch.tokenizer import MyTokenizer

t_tokenizer = MyTokenizer(
    vocab_file,
    merges_file,
)

#sorted(vars(t_tokenizer).keys())

['_additional_special_tokens',
 '_bos_token',
 '_cls_token',
 '_decode_use_source_tokenizer',
 '_eos_token',
 '_mask_token',
 '_pad_token',
 '_pad_token_type_id',
 '_sep_token',
 '_unk_token',
 'added_tokens_decoder',
 'added_tokens_encoder',
 'bpe_ranks',
 'cache',
 'decoder',
 'deprecation_warnings',
 'encoder',
 'init_inputs',
 'init_kwargs',
 'lower',
 'merges_file',
 'model_input_names',
 'model_max_length',
 'name_or_path',
 'normalization',
 'padding_side',
 'special_puncts',
 'unique_no_split_tokens',
 'verbose',
 'vocab_file']

In [12]:
t_tokenizer.save_pretrained("./mytokenizer")

('./mytokenizer/tokenizer_config.json',
 './mytokenizer/special_tokens_map.json',
 './mytokenizer/vocab.txt',
 './mytokenizer/bpe.codes',
 './mytokenizer/added_tokens.json')

In [16]:
MyTokenizer.from_pretrained("./mytokenizer")

() {'normalization': True, 'bos_token': '<s>', 'eos_token': '</s>', 'sep_token': '</s>', 'cls_token': '<s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'mask_token': '<mask>', 'lower': False, 'special_tokens_map_file': './mytokenizer/special_tokens_map.json', 'tokenizer_file': None, 'name_or_path': './mytokenizer'}


TypeError: __init__() missing 2 required positional arguments: 'vocab_file' and 'merges_file'

Qué garcha pasa?

In [14]:

from transformers import AutoTokenizer

tugo = AutoTokenizer.from_pretrained("vinai/bertweet-base")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


() {'model_max_length': 128, 'vocab_file': '/home/jmperez/.cache/huggingface/transformers/9a877d0d57efbfeae96fec396a35595dc8c4685fe2b7b2049c6c094e24a0e8bf.f8a4dfe5c3c45a26f9df849d732decb191dc0c05ab270799695430332d143982', 'merges_file': '/home/jmperez/.cache/huggingface/transformers/1c2d05a06ac61a063ad62a7590731a28cc62f58e2802c76b5f993165f25894a9.75877d86011e5d5d46614d3a21757b705e9d20ed45a019805d25159b4837b0a4', 'special_tokens_map_file': None, 'tokenizer_file': None, 'name_or_path': 'vinai/bertweet-base'}


In [15]:

tugo.vocab_file

'/home/jmperez/.cache/huggingface/transformers/9a877d0d57efbfeae96fec396a35595dc8c4685fe2b7b2049c6c094e24a0e8bf.f8a4dfe5c3c45a26f9df849d732decb191dc0c05ab270799695430332d143982'

In [41]:

tugo.save_pretrained("./pepe")

('./pepe/tokenizer_config.json',
 './pepe/special_tokens_map.json',
 './pepe/vocab.txt',
 './pepe/bpe.codes',
 './pepe/added_tokens.json')

## WordPiece

In [9]:


tokenizer = BertWordPieceTokenizer(
    clean_text=True,
    handle_chinese_chars=True,
    strip_accents=False,
    lowercase=False,
)



In [10]:
from finetune_vs_scratch.preprocessing import special_tokens

tokenizer.add_special_tokens(special_tokens)

4

In [11]:
tokenizer.train_from_iterator(
    tweets,
    vocab_size=40_000,
    min_frequency=2,
    show_progress=True,

    special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"] + special_tokens,
    limit_alphabet=600,
    wordpieces_prefix="##",
)

In [12]:
!mkdir test_tokenizer
tokenizer.save_model("./test_tokenizer/")

mkdir: cannot create directory ‘test_tokenizer’: File exists


['./test_tokenizer/vocab.txt']

## Reload

In [15]:
from transformers import BertTokenizer, BertTokenizerFast

bert_tokenizer = BertTokenizerFast.from_pretrained(
    "./test_tokenizer/",
    never_split=special_tokens,
)


## Loading trained tokenizers

In [18]:

!pip freeze | grep transf

transformers @ file:///home/jmperez/.cache/pypoetry/artifacts/d7/d1/6f/c08a86d09ebb99ef7233126f12dce131f0bcf38f23b1c8aa8d2065c528/transformers-4.8.2-py3-none-any.whl


In [20]:
from transformers import BertTokenizerFast, AutoTokenizer, BertTokenizer
from finetune_vs_scratch.model import load_tokenizer

bert_tokenizer = BertTokenizer.from_pretrained(
    "../models/tokenizers/betito_cased_accents/",
    never_split=["@usuario"]
)


bert_tokenizer("@usuario")

{'input_ids': [2, 5, 3], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}

Si es Fast, le tenemos que poner `additional_special_tokens`, LTA

In [24]:
from transformers import BertTokenizerFast, AutoTokenizer, BertTokenizer
from finetune_vs_scratch.model import load_tokenizer

bert_tokenizer = BertTokenizerFast.from_pretrained(
    "../models/tokenizers/betito_cased_accents/",
    additional_special_tokens=["@usuario"]
)


bert_tokenizer("@usuario")

{'input_ids': [2, 5, 3], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}

In [15]:

bert_tokenizer("@usuario")

{'input_ids': [2, 38, 1164, 3], 'token_type_ids': [0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1]}

In [3]:
import re 

tweet = "@usuario @usuario jajaja gil emoji riendo emoji emoji riendo emoji"

tweet = re.sub("emoji.*?emoji", "emoji", tweet)
print(tweet)
tokens = bert_tokenizer(tweet)["input_ids"]
print(tokens)
print(bert_tokenizer.decode(tokens))

@usuario @usuario jajaja gil emoji emoji
[2, 38, 1164, 38, 1164, 5733, 621, 5411, 8, 8, 3]
[CLS] @ usuario @ usuario jajaja gil emoji emoji [SEP]


In [25]:

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', use_fast=False, do_lower_case=True, do_basic_tokenize=True)
tokenizer.add_tokens(['graft', 'grafts'])

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

2

In [28]:
tokenizer.convert_ids_to_tokens(tokenizer.encode("graft"))

['[CLS]', 'graft', '[SEP]']